# PySpark DataFrames  
  
[Home](../../README.md)  

![](https://miro.medium.com/max/2560/1*qgkjkj6BLVS1uD4mw_sTEg.png)

# DataFrame Basics  
  
### Key Points    
  
| ___ | Navigation| Links | ___ | 
|-------|-----------|---------|----------|
|[First DataFrame](#First-DataFrame)|[Amending Schema](#Amending-Schema) |[Select Operations](#Select-Operations) |[Column operations](#Column-operations)|
| [SQL](#SQL)|[DataFrame Filter Operations](#DataFrame-Filter-Operations) | | |
| | | | |

**Setting up**  

```python
spark = SparkSession.builder.appName('Basics').getOrCreate() 
df = spark.read.json(peopleFile)
```  
  
**Useful commands**. 
  
```python  
df.show()
df.printSchema()
df.describe().show()
df.select('column').show()
```

## First DataFrame

In [2]:
import findspark
findspark.init('/opt/apache-spark/spark-2.4.7-bin-hadoop2.7')
from pyspark.sql import SparkSession
peopleFile = 'in/df/people.json'

In [3]:
spark = SparkSession.builder.appName('Basics').getOrCreate() 

In [4]:
df = spark.read.json(peopleFile)

In [5]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [6]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [7]:
df.columns # don't need ()

['age', 'name']

In [8]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [9]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



## Amending Schema

In [10]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

In [11]:
# struct field takes three inputs, name, datatype, nullable
print("Defining table dataschema")
data_schema = [  StructField('age',IntegerType(),True),  
                 StructField('name', StringType(),True)
              ]

Defining table dataschema


In [12]:
final_struc = StructType(fields=data_schema)

In [13]:
df = spark.read.json(peopleFile,schema=final_struc)

In [14]:
df.printSchema() # now our age schema has been udpated

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



## Select Operations

In [18]:
print(type(df['age']))
print(type(df.select('age')))

<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [24]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [27]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [28]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Column operations

#### Adding a Column

In [47]:
df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [31]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [46]:
 df.withColumn('DoubleAge',df['age']*2).show()

+----+-------+---------+
| age|   name|DoubleAge|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



#### Renaming a Column

In [48]:
df.withColumnRenamed('age','my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



# SQL

#### Crete a view from Dataframe

In [49]:
df.createOrReplaceTempView('people')

#### Run Queries

In [52]:
results = spark.sql("SELECT * FROM people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [53]:
new_results = spark.sql("SELECT * FROM people where age =30").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



# DataFrame Filter Operations

In [56]:
from pyspark.sql import SparkSession

In [57]:
spark = SparkSession.builder.appName('ops').getOrCreate()

In [60]:
df = spark.read.csv('in/df/appl_stock.csv',inferSchema=True,header=True)

In [61]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [62]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    